In [25]:
import warnings

import numpy as np
import numpy.linalg
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler, minmax_scale
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA 

from scipy.stats import randint, loguniform
import pickle
import os
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

sns.set(style="whitegrid")

%matplotlib inline

In [ ]:
notebooks_dir = os.path.dirname(os.path.abspath(__file__))

base_dir = os.path.abspath(os.path.join(notebooks_dir, ".."))
csv_file = os.path.join(base_dir, "data", "processed", "musicgenre_3sec.csv")
labelencoder_path = os.path.join(base_dir,"data", "processed", "labelencoder.pkl")
standardscaler_path = os.path.join(base_dir,"data", "processed", "standardscaler.pkl")
pca_path = os.path.join(base_dir,"data", "processed", "pca.pkl")
best_svm_path = os.path.join(base_dir, "data", "processed","best_svm.pkl")

# Loading the Data

In [ ]:
df = pd.read_csv(csv_file)

In [29]:
df.head()

,filename,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,spectral_rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829707,1972.334258,117272.640573,3714.063439,...,39.547077,-3.230046,36.606857,0.696385,37.766132,-5.035945,33.668549,-0.239585,43.818882,blues
1,blues.00000.1.wav,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297514,2010.751494,65940.666037,3870.510442,...,64.819786,-6.025473,40.548813,0.127131,51.048943,-2.808956,97.221497,5.771882,60.360352,blues
2,blues.00000.2.wav,0.347746,0.092495,0.130895,0.004552,1790.722358,110071.206762,2088.184750,73391.498088,4000.206581,...,68.306786,-1.714475,28.136944,2.329553,47.211426,-1.925621,52.922432,2.466996,33.163998,blues
3,blues.00000.3.wav,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936309,1967.920582,79805.901501,3579.149639,...,48.543201,-3.786986,28.419542,1.153315,35.682701,-3.501979,50.610344,3.580636,32.325878,blues
4,blues.00000.4.wav,0.335481,0.088482,0.142370,0.001734,1634.465076,77425.419156,1954.633566,57359.695597,3480.096905,...,30.829538,0.635798,44.645554,1.591107,51.415867,-3.364908,26.421089,0.501504,29.109531,blues


In [31]:
df = df.drop(['filename'], axis = 1)

In [32]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

### Label Encoding

In [33]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

### PCA (Principal Component Analysis)

In [34]:
X_mean = np.mean(X,axis=0)
X_std = np.std(X,axis=0)

In [35]:
Z = (X - X_mean) / X_std

In [36]:
Z

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,spectral_rolloff_mean,spectral_rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,-0.488830,0.632510,0.001747,0.249046,-0.570428,-0.567710,-0.502775,-0.010621,-0.522484,-0.364570,...,-0.752509,-0.300426,0.170132,-0.419434,-0.004960,-0.382801,-0.497188,-0.509245,0.131495,-0.287560
1,-0.400922,0.196737,-0.263448,-0.331804,-0.511941,-0.749099,-0.431787,-0.522834,-0.427036,-0.641932,...,0.458408,0.435445,-0.323131,-0.310798,-0.114953,-0.033727,-0.060999,1.022522,1.276824,0.069513
2,-0.354330,0.787387,0.012109,0.528064,-0.547287,-0.704596,-0.288706,-0.448486,-0.347908,-0.565132,...,0.577203,0.536976,0.437560,-0.652855,0.310605,-0.134577,0.112015,-0.045184,0.647164,-0.517562
3,-0.176528,0.240692,0.018774,-0.093820,-0.720776,-0.705920,-0.510931,-0.384481,-0.604795,-0.485964,...,-0.503183,-0.038484,0.071858,-0.645067,0.083329,-0.437554,-0.196738,-0.100910,0.859339,-0.535654
4,-0.489653,0.372482,0.180513,-0.263522,-0.755533,-0.779854,-0.535482,-0.608459,-0.665226,-0.698080,...,0.109592,-0.554256,0.852274,-0.197896,0.167920,-0.024084,-0.169890,-0.683925,0.272690,-0.605083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986,-0.342918,-0.447335,-1.179786,-0.723244,-0.930420,-0.581615,-0.958849,-0.327896,-0.943133,-0.523814,...,0.739642,-0.159234,-0.862896,-0.355411,-0.954244,-0.551199,-0.623713,-0.174829,0.505811,-0.398288
9987,-0.080616,-0.232255,-1.063331,-0.723665,-0.464679,-0.319234,-0.609676,-0.209138,-0.497674,-0.313201,...,0.105809,-0.516254,-1.434498,0.448434,-0.744427,0.018322,-1.844812,0.212610,0.250403,-0.826192
9988,-0.353574,0.440533,-1.147121,-0.561409,-1.136798,0.568582,-1.251826,0.195191,-1.296427,0.658416,...,-0.395465,0.855061,0.266612,-0.799084,0.816368,-0.694439,0.825837,-0.170397,0.136204,-0.340258
9989,0.076536,-0.023823,-0.940757,-0.662933,-0.153357,-0.495597,-0.412584,-0.973540,-0.154233,-0.770243,...,0.473292,-0.638854,-0.204973,-0.956260,1.100440,-0.798838,0.948919,-0.719842,0.304070,-0.961958


In [37]:
cov_matrix = np.cov(Z.T)

In [38]:
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
print('Eigen values:\n', eigenvalues)
print('Eigen values Shape:', eigenvalues.shape)
print('Eigen Vector Shape:', eigenvectors.shape)

Eigen values:
 [1.13519035e+01 7.70167716e+00 5.94126907e+00 3.77038107e+00
 2.49363038e+00 2.09900379e+00 1.68593060e+00 1.51637247e+00
 1.41769571e+00 1.22513523e+00 1.06327082e+00 9.92938269e-01
 9.75434900e-01 8.46953051e-01 7.80129962e-01 6.70376272e-01
 6.35917737e-01 3.48022202e-03 7.74593806e-03 1.20081613e-02
 6.15046780e-01 3.51461196e-02 5.39500655e-02 7.24302528e-02
 1.02669793e-01 5.55828939e-01 1.42352616e-01 5.20369373e-01
 1.68874003e-01 1.81486798e-01 1.99876041e-01 4.95899242e-01
 4.84032762e-01 4.81175670e-01 2.11460289e-01 4.61094520e-01
 2.28898251e-01 2.34575120e-01 2.44129208e-01 2.62716520e-01
 2.56290394e-01 2.57442606e-01 4.46683937e-01 2.87345099e-01
 2.98805841e-01 3.10627999e-01 4.32071976e-01 4.28681632e-01
 4.18495239e-01 3.30377377e-01 3.98587019e-01 3.88368191e-01
 3.85434042e-01 3.43431145e-01 3.51154619e-01 3.59837707e-01
 3.68804226e-01]
Eigen values Shape: (57,)
Eigen Vector Shape: (57, 57)


In [39]:
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

In [40]:
explained_var = np.cumsum(eigenvalues) / np.sum(eigenvalues)
explained_var

array([0.19913627, 0.33423989, 0.43846225, 0.50460266, 0.54834618,
       0.58516713, 0.6147419 , 0.64134226, 0.66621162, 0.68770307,
       0.70635508, 0.72377331, 0.74088449, 0.75574182, 0.76942694,
       0.78118675, 0.79234209, 0.8031313 , 0.81288171, 0.82201009,
       0.8307092 , 0.83920016, 0.84764099, 0.85572956, 0.86356533,
       0.87114478, 0.87866476, 0.88600605, 0.8929981 , 0.8998109 ,
       0.90657222, 0.91304182, 0.91935413, 0.92551413, 0.93153863,
       0.93733414, 0.94278321, 0.9480249 , 0.95306553, 0.95767413,
       0.96219022, 0.96668609, 0.97096863, 0.97508357, 0.97909893,
       0.98280839, 0.98631463, 0.98949829, 0.9924607 , 0.99495786,
       0.99675891, 0.99802949, 0.99897588, 0.99959242, 0.99980307,
       0.99993895, 1.        ])

In [42]:
n_components = np.argmax(explained_var >= 0.95) + 1
n_components

39

In [43]:
ss = StandardScaler()
ss.fit(X)

StandardScaler()

In [44]:
Z = ss.transform(X)

In [45]:
pca = PCA(n_components=n_components)
pca.fit(Z)
x_pca = pca.transform(Z)

df_pca = pd.DataFrame(x_pca,
                       columns=['PC{}'.
                       format(i+1)
                        for i in range(39)])
df_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39
0,-1.843266,0.418261,0.642602,-1.016014,0.023422,-0.937471,0.187564,-0.369927,-0.212065,0.333194,...,-0.121577,1.009547,-0.657064,0.377374,0.375329,-0.308822,-0.040978,0.512895,0.074727,-0.327283
1,-2.651712,0.289402,1.621585,0.374095,-0.418447,-0.235988,-0.555424,-0.612678,-0.368074,0.659386,...,0.225862,0.227193,-0.028698,0.168344,-0.379947,-0.147613,0.507101,0.049298,0.571405,0.105790
2,-1.811727,0.119079,0.473266,-0.142429,-1.220585,-1.059546,-0.197180,-0.568976,-0.453595,0.004732,...,-0.205195,0.157112,0.506498,0.397865,-0.415143,-0.138652,0.640980,0.669590,0.504204,0.043471
3,-2.405584,0.245485,0.827622,-0.727531,-0.162034,-0.922470,-0.076509,-0.772914,-0.052168,0.531534,...,0.075773,0.710916,0.003214,0.702802,0.653944,-0.463438,0.858189,-0.002855,0.216467,-0.231620
4,-2.989091,0.211797,0.592575,-0.323338,-0.613097,-1.172347,0.045194,-0.651148,-0.130061,0.173264,...,-0.389293,-0.030481,-0.105732,0.572333,0.319476,-0.616089,0.422812,-0.050865,-0.351306,-0.206565


In [46]:
Z = df_pca.to_numpy()

### Creating Train, Test and Validation Set

In [47]:
X_train, X_test, y_train, y_test = train_test_split(Z,y,test_size = 0.2, random_state = 42,stratify=y)

# Training Models

### Support Vector Machine (SVM)

In [48]:
param_dist_svm = {
    'C': loguniform(1e-4, 1e+1),
    'kernel': ['linear', 'rbf'], 
}

svm = SVC(random_state=42)
random_search_svm = RandomizedSearchCV(
    svm, param_distributions=param_dist_svm, n_iter=12,  
    scoring='accuracy', n_jobs=-1, random_state=42  
)

random_search_svm.fit(X_train, y_train)

best_svm = random_search_svm.best_estimator_
y_pred_svm = best_svm.predict(X_test)
test_accuracy_svm = accuracy_score(y_test, y_pred_svm)

y_train_pred_svm = best_svm.predict(X_train)
train_accuracy_svm = accuracy_score(y_train, y_train_pred_svm)

print("Train SVM Accuracy:", train_accuracy_svm)
print("Test SVM Accuracy:", test_accuracy_svm)

Train SVM Accuracy: 0.924049049049049
Test SVM Accuracy: 0.848424212106053


### K-Nearest Neighbors (KNN)

In [49]:
param_grid = {
    'n_neighbors': randint(1, 100,5), 
    'weights': ['uniform', 'distance'],  
    'p': [1, 2]  
}

knn = KNeighborsClassifier()

random_search_knn = RandomizedSearchCV(
    knn, param_distributions=param_grid, n_iter=20, cv=5, random_state=42
)
random_search_knn.fit(X_train, y_train)

best_knn = random_search_knn.best_estimator_
y_pred_knn = best_knn.predict(X_test)
test_accuracy_knn = accuracy_score(y_test, y_pred_knn)

y_train_pred_knn = best_knn.predict(X_train)
train_accuracy_knn = accuracy_score(y_train, y_train_pred_knn)

print("Train KNN Accuracy:", train_accuracy_knn)
print("Test KNN Accuracy:", test_accuracy_knn)

Train KNN Accuracy: 0.9055305305305306
Test KNN Accuracy: 0.8319159579789895


### Decision Tree

In [50]:
param_dist_dt = {
    'max_depth': randint(1, 51), 
    'min_samples_split': randint(2, 11), 
    'min_samples_leaf': randint(1, 11),  
    'criterion': ['gini', 'entropy'],  
    
}

dt = DecisionTreeClassifier(random_state=42)
random_search_dt = RandomizedSearchCV(
    dt, param_distributions=param_dist_dt, n_iter=50, 
    scoring='accuracy', n_jobs=-1, random_state=42
)

random_search_dt.fit(X_train, y_train)

best_dt = random_search_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)
test_accuracy_dt = accuracy_score(y_test, y_pred_dt)

y_train_pred_dt = best_dt.predict(X_train)
train_accuracy_dt = accuracy_score(y_train, y_train_pred_dt)

print("Train DT Accuracy:", train_accuracy_dt)
print("Test DT Accuracy:", test_accuracy_dt)

Train DT Accuracy: 0.8267017017017017
Test DT Accuracy: 0.5647823911955978


### Random Forest

In [51]:
param_dist_rf = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [8, 12, 16, 20, 24, 28, 32],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


rf = RandomForestClassifier(random_state=42)
random_search_rf = RandomizedSearchCV(
    rf, param_distributions=param_dist_rf, n_iter=50,  
    scoring='accuracy', n_jobs=-1, random_state=42  
)

random_search_rf.fit(X_train, y_train)

best_rf = random_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_pred_rf)

y_train_pred_rf = best_rf.predict(X_train)
train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)

print("Train RF Accuracy:", train_accuracy_rf)
print("Test RF Accuracy:", test_accuracy_rf)

Train RF Accuracy: 0.998998998998999
Test RF Accuracy: 0.7903951975987994


## Pickle the Objects

In [ ]:
# Save the LabelEncoder
with open(labelencoder_path, "wb") as file:
    pickle.dump(labelencoder, file)

# Save the StandardScaler
with open(standardscaler_path, "wb") as file:
    pickle.dump(ss, file)

# Save the PCA object
with open(pca_path, "wb") as file:
    pickle.dump(pca, file)

# Save the best SVM estimator
with open(best_svm_path, "wb") as file:
    pickle.dump(best_svm, file)